### Goals
- Build a movie recommender system

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Loading dataset
movies_df = pd.read_csv("./ml-latest/movies.csv")
# links_df = pd.read_csv("./ml-latest/links.csv")
ratings_df = pd.read_csv("./ml-latest/ratings.csv")
# tags_df= pd.read_csv("./ml-latest/tags.csv")
movies_df.head()
print(ratings_df.head())

   userId  movieId  rating   timestamp
0       1      169     2.5  1204927694
1       1     2471     3.0  1204927438
2       1    48516     5.0  1204927435
3       2     2571     3.5  1436165433
4       2   109487     4.0  1436165496


In [5]:
movies_df[movies_df["title"] == "Gladiator (2000)"]

,movieId,title,genres
3488,3578,Gladiator (2000),Action|Adventure|Drama


In [6]:
print(movies_df.shape)
print(movies_df.size)
# print(movies_df.dtypes)
# print(movies_df.info())
# print(movies_df.describe())
print(ratings_df.shape)
print(ratings_df.size)
# print(ratings_df.dtypes)

(34208, 3)
102624
(22884377, 4)
91537508


In [7]:
temp_df = movies_df.head().copy()
temp_df["title"]
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [8]:
# Preprocessing
# Collaborative filtering
# Constest based filtering
# Movie dataset from Kaggle
# Preprocessing
# cleaning the data
# Understand how to do collaborative filtering
# By using similarity score 
# print(movies_df["title"])
# Cleaning movies data
movies_df["year"] = movies_df.title.str.extract("(\(\d{4}\))", expand=False)
movies_df["year"] = movies_df.year.str.extract("(\d{4})", expand=False)
movies_df["title"] = movies_df.title.str.replace(r"(\(\d{4}\))", "", regex=True)
movies_df["title"] = movies_df["title"].apply(lambda x: x.strip())
movies_df = movies_df.drop("genres", axis=1)
movies_df

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
...,...,...,...
34203,151697,Grand Slam,1967
34204,151701,Bloodmoney,2010
34205,151703,The Butterfly Circus,2009
34206,151709,Zero,2015


In [9]:
# ratings data cleaning
ratings_df= ratings_df.drop("timestamp", axis=1)
ratings_df.head()


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


- Collaborating filtering to predict ratings
- Using Pearson Correlation Function

$$

$$

In [10]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [11]:
# find the row present in the movies based on the title present in the userInput
# then add that data to input
# removed not required field
inputMovies = pd.DataFrame(userInput)
inputMovies = movies_df[movies_df["title"].isin(inputMovies["title"].tolist())]
inputMovies

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
293,296,Pulp Fiction,1994
1246,1274,Akira,1988
1885,1968,"Breakfast Club, The",1985


In [12]:
# 